In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [2]:
# Example Bloom Filter

from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = double_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [3]:
# Example Bloom Filter 2

from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = double_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
h = double_sha256(b'goodbye')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [4]:
# Example Bloom Filter 3

from helper import double_sha256, hash160

bit_field_size = 10
bit_field = [0] * bit_field_size

phrase1 = b'hello world'
h1 = double_sha256(phrase1)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase1)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
phrase2 = b'goodbye'
h1 = double_sha256(phrase2)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase2)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [5]:
# Example BIP0037 Bloom Filter

from helper import murmur3
from bloomfilter import BIP37_CONSTANT

field_size = 2
num_functions = 2
tweak = 42

bit_field_size = field_size * 8
bit_field = [0] * bit_field_size

for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Exercise 1

#### 1.1 Given a Bloom Filter with these parameters: size=10, function count=5, tweak=99, which bits are set after adding these items? 

 * `b'Hello World'`
 * `b'Goodbye!'`

#### 1.2. Make [this test](/edit/session8/bloomfilter.py) pass.

In [6]:
# Exercise 1.1
from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')

# bit_field_size is 8 * field_size
bit_field_size = field_size * 8
# create a bit field with the appropriate size
bit_field = [0] * bit_field_size

# for each item you want to add to the filter
for item in items:
    # iterate function_count number of times
    for i in range(function_count):
        # BIP0037 spec seed is i*BIP37_CONSTANT + tweak
        seed = i * BIP37_CONSTANT + tweak
        # get the murmur3 hash given that seed
        h = murmur3(item, seed=seed)
        # set the bit to be h mod the bit_field_size
        bit = h % bit_field_size
        # set the bit_field at the index bit to be 1
        bit_field[bit] = 1
# print the bit field converted to bytes using bit_field_to_bytes in hex
print(bit_field_to_bytes(bit_field).hex())

4000600a080000010940


In [7]:
# Exercise 1.2

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_add'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 2

#### 2.1. Make [this test](/edit/session8/bloomfilter.py) pass.

In [8]:
# Exercise 2.1

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_filterload'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 3

#### 3.1. Make [this test](/edit/session8/merkleblock.py) pass.

In [9]:
# Exercise 3.1

reload(merkleblock)
run_test(merkleblock.MerkleBlockTest('test_parse'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 4

#### 4.1. Make [this test](/edit/session8/network.py) pass.

In [10]:
# Exercise 4.1

reload(network)
run_test(network.GetDataMessageTest('test_serialize'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 5

#### 5.1. Make [this test](/edit/session8/merkleblock.py) pass.

In [11]:
reload(merkleblock)
run_test(merkleblock.MerkleBlockTest('test_is_valid'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 6

#### 6.1. You have been sent some unknown amount of testnet bitcoins to your address. 

Send all of it back (minus fees) to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv` using only the networking protocol.

In [12]:
# Exercise 6.1

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import double_sha256, little_endian_to_int, encode_varint, read_varint, decode_base58, p2pkh_script, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from tx import Tx, TxIn, TxOut

last_block_hex = '0000000000000002ff6a9878658efe0155882b46d8cc93007b3a224afc27bd29'

secret = little_endian_to_int(double_sha256(b''))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
h160 = decode_base58(addr)

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis

# connect to tbtc.programmingblockchain.com in testnet mode
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)

# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
bf = BloomFilter(30, 5, 90210)
# add the h160 to the bloom filter
bf.add(h160)

# complete the handshake
node.handshake()
# load the bloom filter with the filterload command
node.send(b'filterload', bf.filterload())

# set start block to last_block from above
start_block = bytes.fromhex(last_block_hex)
# send a getheaders message with the starting block
getheaders_message = GetHeadersMessage(start_block=start_block)
node.send(getheaders_message.command, getheaders_message.serialize())

# wait for the headers message
headers_envelope = node.wait_for_commands({HeadersMessage.command})

# get the stream from the headers
stream = headers_envelope.stream()
# parse the headers message
headers = HeadersMessage.parse(stream)
# store the last block as None
last_block = None
# initialize the GetDataMessage
get_data_message = GetDataMessage()
# loop through the blocks in the headers
for b in headers.blocks:
    # check that the proof of work on the block is valid
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    # check that this block's prev_block is the last block
    if last_block is not None and b.prev_block != last_block:
        raise RuntimeErrer('chain broken')
    # add a new item to the get_data_message
    # should be FILTERED_BLOCK_DATA_TYPE and block hash
    get_data_message.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
    # set the last block to the current hash
    last_block = b.hash()
# send the getdata message
node.send(get_data_message.command, get_data_message.serialize())

# initialize prev_tx, prev_index and pt to None
prev_tx, prev_index, prev_tx_obj = None, None, None
# loop while prev_tx is None 
while prev_tx is None:
    # wait for the merkleblock or tx commands
    envelope = node.wait_for_commands({b'merkleblock', b'tx'})
    # initialize the stream from the envelope
    stream = envelope.stream()
    # if we have the merkleblock command
    if envelope.command == b'merkleblock':
        # parse the MerkleBlock
        mb = MerkleBlock.parse(stream)
        # check that the MerkleBlock is valid
        if not mb.is_valid():
            raise RuntimeError('invalid merkle proof')
    # else we have the tx command
    else:
        # parse the tx
        prev_tx_obj = Tx.parse(stream, testnet=True)
        # loop through the tx outs
        for i, tx_out in enumerate(prev_tx_obj.tx_outs):
            # if our output has the same address as our address we found it
            if tx_out.script_pubkey.address(testnet=True) == addr:
                # we found our utxo. set prev_tx, prev_index, and transaction
                prev_tx = prev_tx_obj.hash()
                prev_index = i
                print('found: {}:{}'.format(prev_tx.hex(), prev_index))
# create tx_in
tx_in = TxIn(prev_tx, prev_index, b'', 0xffffff)
# set the cache of the previous tx to prev_tx_obj so we don't talk to a block explorer
tx_in.cache[prev_tx] = prev_tx_obj
# create the tx_ins array
tx_ins = [tx_in]
# calculate how much is in this UTXO
total = prev_tx_obj.tx_outs[prev_index].amount
# create a new transaction out to the right address with the right amount (subtract fee)
tx_outs = [TxOut(total-fee, target_script)]
# create a new transaction
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
# sign the transaction
tx_obj.sign_input(0, private_key, SIGHASH_ALL)
# serialize and hex to see what it looks like
print(tx_obj.serialize().hex())
# send this signed transaction on the network
node.send(b'tx', tx_obj.serialize())
# wait a sec so this message goes through to the other node time.sleep(1) 
time.sleep(1)
# now ask for this transaction from the other node
# create a GetDataMessage
get_data_message = GetDataMessage()
# ask for our transaction by adding it to the message
get_data_message.add_data(TX_DATA_TYPE, tx_obj.hash())
# send the message
node.send(get_data_message.command, get_data_message.serialize())
# now wait for a response
envelope = node.wait_for_commands({b'tx', b'reject'})
if envelope.command == b'tx':
    print(envelope.payload.hex())
else:
    print(envelope.payload)

mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv
sending: version: 7f11010000000000000000001a8faa5b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d20da32ae4800a5697a1b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
receiving: version: 7f1101000d040000000000001c8faa5b00000000000000000000000000000000000000000000ffff4830fd33cabd0d040000000000000000000000000000000000000000000000009e0e3c9ec75106a3102f5361746f7368693a302e31362e332fad94150001
sending: verack: 
receiving: verack: 
sending: filterload: 1e000000000448000000000004000000000200000000000000000000000000050000006260010001
sending: getheaders: 7f1101000129bd27fc4a223a7b0093ccd8462b885501fe8e6578986aff02000000000000000000000000000000000000000000000000000000000000000000000000000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: e46ed5a9cf7b7d24
sending: pong: e46ed5a9cf7b7d24
receiving: addr: 011c8